<a href="https://colab.research.google.com/github/jinju-yang/Shinhan-Bigdata-Hackerthon/blob/1-4%2Fchildren-age-clustering/1_4_%EC%9E%90%EB%85%80_%EB%82%98%EC%9D%B4%EB%8C%80_%ED%81%B4%EB%9F%AC%EC%8A%A4%ED%84%B0%EB%A7%81_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
from sklearn.cluster import KMeans
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler

In [ ]:
df_asset = pd.read_csv('asset_cluster.csv')
df_asset

In [ ]:
df_asset.rename(columns={"pk1": "기준년월", "pk2": "성별", "pk3": "연령대", "pk4": "지역"}, inplace=True)

In [ ]:
df_asset.rename(columns={'cb02m': '이용금액_마트', 'cc01m': '이용금액_식당',
                          'cd07m': '이용금액_서점',
                          'ce01m': '이용금액_학원학습지', 'cf11m': '이용금액_놀이동산',
                          'cf16m': '이용금액_전시/관람/체험', 'cf17m': '이용금액_오락실/PC방', 'la04r': '최근1년보험비율_어린이',
                          'lb04r': '보험비율_어린이', 'lb12m': '보험월납평균_어린이',
                          'lc05r': '보험금지급경험비율_어린이', 'ld05r': '보험대출경험비율_어린이'}, inplace=True)

In [ ]:
columns = ['연령대_5', '이용금액_서점', '이용금액_학원학습지', '최근1년보험비율_어린이']

In [ ]:
def extract_age(row):
    age = int(row[:2])  # Extract the first two characters as an integer
    return age if row.endswith('s') else age + 5

df_asset['연령대_5'] = df_asset['연령대'].apply(extract_age)

df_asset.sample(3)

In [ ]:
# 60대는 제외 -> 서비스 대상에 적합하지 않다.
df_asset = df_asset[df_asset['연령대_5']<60]

In [ ]:
df_asset['연령대_5'].unique()

In [ ]:
df_asset[columns].isnull().sum()

In [ ]:
# 결측치 제거
## 어린이보험이 null이면 자녀가 없음을 의미
df_asset.dropna(subset = columns, inplace=True)

In [ ]:
## 어린이보험비율=0인 행들 삭제
### 자녀가 없다고 판단
df_asset = df_asset[df_asset['보험비율_어린이']!=0]

In [ ]:
asset_high = df_asset[df_asset['cluster']==2]
asset_midd = df_asset[df_asset['cluster']==0]
asset_low = df_asset[df_asset['cluster']==1]

In [ ]:
asset_high.head()

In [ ]:
scaler = MinMaxScaler()
asset_high[columns] = scaler.fit_transform(asset_high[columns])

In [ ]:
asset_high[columns].head()

In [ ]:
#컬럼 설정
X_high = asset_high[columns]

In [ ]:
# Elbow Method로 최적의 k값 찾기
sse = []
k_range = range(1, 11)  # k의 범위 설정 (1부터 10까지)

for k in k_range:
    kmeans = KMeans(n_clusters=k, random_state=42)
    kmeans.fit(X_high)
    sse.append(kmeans.inertia_)  # 군집 내 거리의 합 (SSE)

# Elbow Plot 그리기
plt.figure(figsize=(10, 6))
plt.plot(k_range, sse, marker='o')
plt.title('Elbow Method for Optimal k')
plt.xlabel('Number of Clusters (k)')
plt.ylabel('Sum of Squared Errors (SSE)')
plt.xticks(k_range)
plt.grid()
plt.show()

In [ ]:
from sklearn.metrics import silhouette_score

silhouette_scores = []
k_range = range(1, 11)  # k의 범위 설정 (1부터 10까지)

# k 값의 범위 설정
for k in k_range:
    kmeans = KMeans(n_clusters=k, random_state=42)
    kmeans.fit(X_high)

    if k > 1:  # 군집이 1개일 경우 실루엣 계수를 계산할 수 없음
        score = silhouette_score(X_high, kmeans.labels_)
        silhouette_scores.append(score)

# Silhouette Score Plot 그리기
plt.figure(figsize=(10, 6))
plt.plot(k_range[1:], silhouette_scores, marker='o', color='g')
plt.title('Silhouette Score for Optimal k')
plt.xlabel('Number of Clusters (k)')
plt.ylabel('Silhouette Score')
plt.grid()
plt.show()

In [ ]:
#k=2로 결정
kmeans = KMeans(n_clusters=2, random_state=42)
asset_high['child_cluster'] = kmeans.fit_predict(X_high)

In [ ]:
cluster_summary = asset_high[columns+['child_cluster']].groupby('child_cluster').mean()
# 원래 스케일로 변환
cluster_summary_original = scaler.inverse_transform(cluster_summary)
cluster_summary_original_df = pd.DataFrame(cluster_summary_original, columns=columns)
cluster_summary_original_df['child_cluster'] = cluster_summary.index
cluster_summary_original_df

In [ ]:
#클러스터 균형 확인
asset_high['child_cluster'].value_counts()

In [ ]:
# 필요한 컬럼만 추출하고 scaling된 데이터를 원래 스케일로 되돌림
scaled_columns = columns  # scaling을 했던 컬럼 리스트
scaled_data = asset_high[scaled_columns]  # scaling된 컬럼들

# 원래 스케일로 변환 (scaler는 원래 스케일링에 사용했던 객체여야 함)
original_data = scaler.inverse_transform(scaled_data)

# 원래 데이터를 asset_high에 복원 (컬럼 이름 유지)
asset_high[scaled_columns] = original_data

# 결과 확인
asset_high.head()

In [ ]:
# 데이터프레임 저장
asset_high.to_csv('asset_high.csv')

In [ ]:
asset_high['이용금액_학원학습지'].describe()

In [ ]:
asset_high['최근1년보험비율_어린이'].describe()

In [ ]:
age_distribution = asset_high['연령대'].value_counts()
plt.figure(figsize=(10, 6))
age_distribution.plot(kind='bar', color='skyblue')
plt.title('Distribution of Age Groups')
plt.xlabel('Age Group')
plt.ylabel('Count')
plt.xticks(rotation=45)
plt.show()

In [ ]:
asset_midd.head()

In [ ]:
scaler = MinMaxScaler()
asset_midd[columns] = scaler.fit_transform(asset_midd[columns])

In [ ]:
asset_midd[columns].head()

In [ ]:
#컬럼 설정
X_midd = asset_midd[columns]

In [ ]:
# Elbow Method로 최적의 k값 찾기
sse = []
k_range = range(1, 11)  # k의 범위 설정 (1부터 10까지)

for k in k_range:
    kmeans = KMeans(n_clusters=k, random_state=42)
    kmeans.fit(X_midd)
    sse.append(kmeans.inertia_)  # 군집 내 거리의 합 (SSE)

# Elbow Plot 그리기
plt.figure(figsize=(10, 6))
plt.plot(k_range, sse, marker='o')
plt.title('Elbow Method for Optimal k')
plt.xlabel('Number of Clusters (k)')
plt.ylabel('Sum of Squared Errors (SSE)')
plt.xticks(k_range)
plt.grid()
plt.show()

In [ ]:
from sklearn.metrics import silhouette_score

silhouette_scores = []
k_range = range(1, 16)  # k의 범위 설정 (1부터 10까지)

# k 값의 범위 설정
for k in k_range:
    kmeans = KMeans(n_clusters=k, random_state=42)
    kmeans.fit(X_midd)

    if k > 1:  # 군집이 1개일 경우 실루엣 계수를 계산할 수 없음
        score = silhouette_score(X_midd, kmeans.labels_)
        silhouette_scores.append(score)

# Silhouette Score Plot 그리기
plt.figure(figsize=(10, 6))
plt.plot(k_range[1:], silhouette_scores, marker='o', color='g')
plt.title('Silhouette Score for Optimal k')
plt.xlabel('Number of Clusters (k)')
plt.ylabel('Silhouette Score')
plt.grid()
plt.show()

In [ ]:
#k=2로 결정
kmeans = KMeans(n_clusters=2, random_state=42)
asset_midd['child_cluster'] = kmeans.fit_predict(X_midd)

In [ ]:
midd_summary = asset_midd[columns+['child_cluster']].groupby('child_cluster').mean()
# 원래 스케일로 변환
midd_summary_original = scaler.inverse_transform(midd_summary)
midd_summary_original_df = pd.DataFrame(midd_summary_original, columns=columns)
midd_summary_original_df['child_cluster'] = midd_summary.index
midd_summary_original_df

In [ ]:
#클러스터 균형 확인
asset_midd['child_cluster'].value_counts()

In [ ]:
# 필요한 컬럼만 추출하고 scaling된 데이터를 원래 스케일로 되돌림
scaled_columns = columns  # scaling을 했던 컬럼 리스트
scaled_data = asset_midd[scaled_columns]  # scaling된 컬럼들

# 원래 스케일로 변환 (scaler는 원래 스케일링에 사용했던 객체여야 함)
original_data = scaler.inverse_transform(scaled_data)

# 원래 데이터를 asset_high에 복원 (컬럼 이름 유지)
asset_midd[scaled_columns] = original_data

# 결과 확인
asset_midd.head()

In [ ]:
asset_midd.shape

In [ ]:
# 데이터프레임 저장
asset_midd.to_csv('asset_midd.csv')

In [ ]:
asset_midd['이용금액_학원학습지'].describe()

In [ ]:
asset_midd['최근1년보험비율_어린이'].describe()

In [ ]:
age_distribution = asset_midd['연령대'].value_counts()
plt.figure(figsize=(10, 6))
age_distribution.plot(kind='bar', color='skyblue')
plt.title('Distribution of Age Groups')
plt.xlabel('Age Group')
plt.ylabel('Count')
plt.xticks(rotation=45)
plt.show()

In [ ]:
asset_low.head()

In [ ]:
scaler = MinMaxScaler()
asset_low[columns] = scaler.fit_transform(asset_low[columns])

In [ ]:
asset_low[columns].head()

In [ ]:
#컬럼 설정
X_low = asset_low[columns]

In [ ]:
# Elbow Method로 최적의 k값 찾기
sse = []
k_range = range(1, 11)  # k의 범위 설정 (1부터 10까지)

for k in k_range:
    kmeans = KMeans(n_clusters=k, random_state=42)
    kmeans.fit(X_low)
    sse.append(kmeans.inertia_)  # 군집 내 거리의 합 (SSE)

# Elbow Plot 그리기
plt.figure(figsize=(10, 6))
plt.plot(k_range, sse, marker='o')
plt.title('Elbow Method for Optimal k')
plt.xlabel('Number of Clusters (k)')
plt.ylabel('Sum of Squared Errors (SSE)')
plt.xticks(k_range)
plt.grid()
plt.show()

In [ ]:
from sklearn.metrics import silhouette_score

silhouette_scores = []
k_range = range(1, 11)  # k의 범위 설정 (1부터 10까지)

# k 값의 범위 설정
for k in k_range:
    kmeans = KMeans(n_clusters=k, random_state=42)
    kmeans.fit(X_low)

    if k > 1:  # 군집이 1개일 경우 실루엣 계수를 계산할 수 없음
        score = silhouette_score(X_low, kmeans.labels_)
        silhouette_scores.append(score)

# Silhouette Score Plot 그리기
plt.figure(figsize=(10, 6))
plt.plot(k_range[1:], silhouette_scores, marker='o', color='g')
plt.title('Silhouette Score for Optimal k')
plt.xlabel('Number of Clusters (k)')
plt.ylabel('Silhouette Score')
plt.grid()
plt.show()

In [ ]:
#k=2로 결정
kmeans = KMeans(n_clusters=2, random_state=42)
asset_low['child_cluster'] = kmeans.fit_predict(X_low)

In [ ]:
low_summary = asset_low[columns+['child_cluster']].groupby('child_cluster').mean()
# 원래 스케일로 변환
low_summary_original = scaler.inverse_transform(low_summary)
low_summary_original_df = pd.DataFrame(low_summary_original, columns=columns)
low_summary_original_df['child_cluster'] = low_summary.index
low_summary_original_df

In [ ]:
#클러스터 균형 확인
asset_low['child_cluster'].value_counts()

In [ ]:
# 필요한 컬럼만 추출하고 scaling된 데이터를 원래 스케일로 되돌림
scaled_columns = columns  # scaling을 했던 컬럼 리스트
scaled_data = asset_low[scaled_columns]  # scaling된 컬럼들

# 원래 스케일로 변환 (scaler는 원래 스케일링에 사용했던 객체여야 함)
original_data = scaler.inverse_transform(scaled_data)

# 원래 데이터를 asset_high에 복원 (컬럼 이름 유지)
asset_low[scaled_columns] = original_data

# 결과 확인
asset_low.head()

In [ ]:
asset_low.shape

In [ ]:
# 데이터프레임 저장
asset_low.to_csv('asset_low.csv')

In [ ]:
asset_low['이용금액_학원학습지'].describe()

In [ ]:
asset_low['최근1년보험비율_어린이'].describe()

In [ ]:
age_distribution = asset_low['연령대'].value_counts()
plt.figure(figsize=(10, 6))
age_distribution.plot(kind='bar', color='skyblue')
plt.title('Distribution of Age Groups')
plt.xlabel('Age Group')
plt.ylabel('Count')
plt.xticks(rotation=45)
plt.show()